In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [ ]:
URL = "https://www.travelandleisure.com/holiday-travel/halloween/most-haunted-places-in-the-world"

In [ ]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)
geolocator = Nominatim(user_agent="geocode_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)

In [ ]:
webpage = requests.get(URL, headers=HEADER)
webpage

In [ ]:
soup = BeautifulSoup(webpage.content, "html.parser")
#soup

In [ ]:
places = soup.find_all("span", attrs = {'class': 'mntl-sc-block-heading__text'})
for i in range(20):
    print(str(i) + "  " + places[i].text)
#print(places[1].text)

In [ ]:
title_list = []
location_list = []
counter = 0

print(places[10].text)
place_list = places[10].text.strip().split(" ")
print(place_list)
for i in range(len(place_list)):
    if place_list[i] == "in":
        counter = 1
        continue
    elif place_list[i] != "in" and counter < 1:
        title_list.append(place_list[i])
    elif place_list[i] != "in" and counter == 1:
        location_list.append(place_list[i])
seperator = " "
title = seperator.join(title_list)
location = seperator.join(location_list)
print(location)
print(title)


location_description = geocoder.geocode(location)
#location_description

In [ ]:
try:
    location_description = geocoder.geocode(location)
except Exception:
    print("No data found for location: " + location)
    #continue
try:
    postcode = location_description[1]['components']['postcode'].replace(" ","-")
except:
    postcode = "Null"
try:
    city = location_description[0]['components']['city']
except:
    city = "Null"
country = location_description[0]['components']['country']
country_code = location_description[0]['components']['country_code']
continent = location_description[0]['components']['continent']
latitude = str(location_description[0]['geometry']['lat'])
longitude = str(location_description[0]['geometry']['lng'])
print(len(location_description))

In [ ]:
city_address = []
for i in range(20):
    
    # SEPERATING TITLE AND LOCATION
    title_list = []
    location_list = []
    counter = 0
    place_list = []
    
    place_list = places[i].text.strip().split(" ")
    #print(place_list)
    for x in range(len(place_list)):
        if place_list[x] == "in":
            counter = 1
            continue
        elif place_list[x] != "in" and counter < 1:
            title_list.append(place_list[x])
        elif place_list[x] != "in" and counter == 1:
            location_list.append(place_list[x])
    seperator = " "
    title = seperator.join(title_list)
    location = seperator.join(location_list)

    # GETTING RAW DATA
    try:
        location_description = geocoder.geocode(location)
    except Exception:
        print(str(i) + " " + "No data found for location: " + location)
        continue
    
    # GETTING ADDRESS VALUES
    try:
        city = location_description[0]['components']['city']
    except Exception:
        try:
            city = location_description[0]['components']['town']
        except Exception:
            try:
                city = location_description[0]['components']['state']
            except Exception:
                city = "NULL"
    country = location_description[0]['components']['country']
    country_code = location_description[0]['components']['country_code']
    continent = location_description[0]['components']['continent']
    latitude = str(location_description[0]['geometry']['lat'])
    longitude = str(location_description[0]['geometry']['lng'])
    
    #GETTING POSTCODE
    lat_long = str(latitude) + ","+ str(longitude) 
    location2 = geolocator.reverse(lat_long, language="en")
    try:
        postcode = location2.raw['address']['postcode'].replace(" ","")
    except:
        postcode = "NULL"
    
    # ADDING VARIABLES TO DICTIONARY
    #print(str(i) +"  "+ title + "  " + city +"  " + country + "  " + country_code.upper() + "  " + continent + "  " + latitude + "  "+ longitude+ "  " + postcode + "\n")
    address_dict = {'Location': title, 'City/State': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode}
    city_address.append(address_dict)
    
print(city_address)

In [ ]:
ca_most_haunted_df = pd.DataFrame.from_dict(city_address)
ca_most_haunted_df